In [1]:
import numpy as np
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.pyplot as plt
import findsources as find
plt.style.use('mplstyle')

In [2]:
from astropy.io import fits
image = 'mosaic_image.fits'  # Change to your file
# image = 'final_mosaicadaptive.fits'  # Change to your file
with fits.open(image) as hdul:
    beam_size = hdul[0].header['BMAJ']* 3600
    print(beam_size)  # Look for BMAJ, BMIN, BPA


40.00000003092


In [3]:
target_ra_hms = '02h00m12s'
target_dec_dms = '-30d53m24s'
search_radius = 20 * 3600 * u.arcsec
flux_thresh = 0.02 # Jy
# Set matching radius (e.g., 45 arcseconds for GLEAM-X resolution)
match_radius = beam_size * u.arcsec 
match_radius = 40 * u.arcsec  # sqrt(40*22)
si = -0.8

# Load GLEAM catalog
gleam_catalog = "GLEAM_EGC_v2.fits.gz"
# gleamx1_catalog = "VIII_110_catalog.dat.gz.fits"
gleamx1_catalog = "VIII_113_catalog2.dat.gz.fits"
gleamx1_catalog = "VIII_113_catalog2glx.dat.gz.fits" # a modified version that gives alpha values
tgss_catalog = "J_A+A_598_A78_table3.dat.gz.fits.gz"
pybdsf_catalog= "mosaic_image.pybdsf.srl.fits"
nvss_catalog= "VIII_65_nvss.dat.gz.fits.gz"



In [4]:
# a modified version that gives alpha values

# with fits.open(gleamx1_catalog, mode="update") as hdul:
#     for i, col in enumerate(hdul[1].columns, start=1):
#         if col.name == 'alpha-SP':
#             print(f"Column index is {i}")

# with fits.open(gleamx2_catalog, mode="update") as hdul:
#     hdr = hdul[1].header
#     hdr['TNULL384'] = '---'
#     hdul.flush()  # Save changes

# with fits.open(gleamx1_catalog) as hdul:
#     print(hdul[1].columns)   # RA in degrees

In [5]:
nu_reference = 147.4
nu_gleam = 151 
nu_gleam_x1 = 151 
# nu_gleam_x1 = 151 
nu_nvss = 1400 

fac_gleam = (nu_reference/nu_gleam)**si
print(fac_gleam)

fac_gleam_x1 = (nu_reference/nu_gleam_x1)**si
print(fac_gleam_x1)

fac_nvss = (nu_reference/nu_nvss)**si
print(fac_nvss)

1.019491410356563
1.019491410356563
6.054864682394719


# GLEAM

In [6]:
names = ['gleam', 'pybdsf']

(pybdsf_ra, pybdsf_dec, gleam_ra, gleam_dec, pybdsf_flux, 
 gleam_flux, pybdsf_flux_err, gleam_flux_err, gleam_flux_151, 
 gleam_flux_151_err, alpha, alpha_err, name) = find.matched_sources_with_alpha(gleam_catalog, pybdsf_catalog, flux_thresh,
                                                                               target_ra_hms, target_dec_dms,
                                                                               search_radius, match_radius, names)
gleam_flux *= (fac_gleam)
gleam_flux_err *= (fac_gleam)

In [7]:
argsort = np.argsort(-pybdsf_flux)
argsort_gleam = np.argsort(-pybdsf_flux)

print(pybdsf_flux[argsort][:10], pybdsf_flux_err[argsort][:10],)
print()
# print(gleam_flux[argsort][:10], gleam_flux_err[argsort][:10],)
print(gleam_flux_151[argsort][:10], gleam_flux_151_err[argsort][:10])

print(alpha[argsort][:10],)
print(alpha_err[argsort][:10],)
print(name[argsort][:10])



[19.07944087 10.08407524  6.2040596   4.66649829  4.07043437  2.63676615
  2.59285157  2.32029264  2.01481443  2.00971007] [0.03169417 0.1228052  0.07585994 0.01961611 0.04961429 0.05010357
 0.01047804 0.02858712 0.01883139 0.04687232]

[17.949156  16.595154  16.595154   4.6637244  3.94791    2.476469
  2.6904912  2.1443262  1.8060807  1.8912518] [0.02400285 0.02889125 0.02889125 0.02368167 0.01800959 0.02056924
 0.01968022 0.01649848 0.01746771 0.02104062]
[-0.862534   -0.80219555 -0.80219555 -0.7282266  -0.59387106 -0.89094305
 -1.1502343  -0.75958085 -0.58621854 -0.6760057 ]
[0.0136427  0.01366115 0.01366115 0.01435938 0.01448209 0.01516625
 0.01472999 0.01566539 0.0171046  0.01651383]
['GLEAM J020012-305324' 'GLEAM J015035-293158' 'GLEAM J015035-293158'
 'GLEAM J015200-294056' 'GLEAM J021527-312144' 'GLEAM J014930-283819'
 'GLEAM J020820-303928' 'GLEAM J020620-332547' 'GLEAM J020311-334722'
 'GLEAM J015853-273607']


In [8]:
print(name[argsort][:10])
print(pybdsf_ra[argsort][:10])
print(pybdsf_dec[argsort][:10])


['GLEAM J020012-305324' 'GLEAM J015035-293158' 'GLEAM J015035-293158'
 'GLEAM J015200-294056' 'GLEAM J021527-312144' 'GLEAM J014930-283819'
 'GLEAM J020820-303928' 'GLEAM J020620-332547' 'GLEAM J020311-334722'
 'GLEAM J015853-273607']
[30.05078099 27.64298912 27.65874067 28.00432669 33.86338507 27.37599285
 32.08494319 31.58750571 30.80088555 29.72171712]
[-30.89120325 -29.54349408 -29.51968    -29.68403569 -31.36275502
 -28.64091047 -30.65952121 -33.4306934  -33.79015013 -27.60315898]


In [9]:
names = ['gleamx1', 'pybdsf']

(
    pybdsf_ra_x1, pybdsf_dec_x1, gleamx1_ra, gleamx1_dec, pybdsf_flux_x1,
    gleamx1_flux, pybdsf_flux_x1_err, gleamx1_flux_err,
    gleamx1_flux_151, gleamx1_flux_151_err, alphax1, alphax1_err, name
) = find.matched_sources_with_alpha(
    gleamx1_catalog, pybdsf_catalog, flux_thresh,
    target_ra_hms, target_dec_dms,
    search_radius, match_radius,
    names
)

gleamx1_flux *= fac_gleam_x1
gleamx1_flux_err *= fac_gleam_x1


# GLEAM-X

In [10]:
argsortx1 = np.argsort(-pybdsf_flux_x1)

print(pybdsf_flux_x1[argsortx1][:10], pybdsf_flux_x1_err[argsortx1][:10],)
print()
# print(gleamx1_flux[argsortx1][:10], gleamx1_flux_err[argsortx1][:10],)
print(gleamx1_flux_151[argsortx1][:10], gleamx1_flux_151_err[argsortx1][:10],)

print('sp index')
print(alphax1[argsortx1][:10],)
print(alphax1_err[argsortx1][:10],)
print()

print(name[argsortx1][:10])


[19.07944087 10.08407524  6.2040596   4.66649829  4.07043437  2.63676615
  2.59285157  2.32029264  2.01481443  2.00971007] [0.03169417 0.1228052  0.07585994 0.01961611 0.04961429 0.05010357
 0.01047804 0.02858712 0.01883139 0.04687232]

[18.8636    9.360348  7.394917  4.915171  3.96779   2.58331   2.944448
  2.400819  1.877183  1.923148] [ 0.003432  0.003294  0.003461  0.002851 -1.        0.002428  0.002628
  0.002273  0.002385  0.002359]
sp index
[-0.7756978   0.          0.         -0.69244562 -0.60566428 -0.81505837
 -1.07143745 -0.72961838 -0.60930943 -0.68338672]
[0.01354298 0.         0.         0.01358501 0.01352334 0.01354934
 0.01359471 0.01357561 0.01356063 0.01358604]

['GLEAM-X J020012.1-305326' 'GLEAM-X J015034.3-293239'
 'GLEAM-X J015038.1-293109' 'GLEAM-X J015201.0-294100'
 'GLEAM-X J021527.3-312147' 'GLEAM-X J014930.3-283825'
 'GLEAM-X J020820.4-303934' 'GLEAM-X J020620.9-332549'
 'GLEAM-X J020312.1-334724' 'GLEAM-X J015853.1-273609']


# TGSS

In [11]:
names = ['tgss', 'pybdsf']

(pybdsf_ra1, pybdsf_dec1, tgss_ra, tgss_dec, pybdsf_flux1, tgss_flux, 
pybdsf_flux1_err, tgss_flux_err) = find.matched_sources_tgss(tgss_catalog, pybdsf_catalog, flux_thresh,
                                                             target_ra_hms, target_dec_dms,
                                                             search_radius, match_radius*1.5, names)

tgss_flux *= 1e-3 # converted to Jy
tgss_flux_err *= 1e-3 # converted to Jy

In [12]:
argsort1 = np.argsort(-pybdsf_flux1)
print(pybdsf_flux1[argsort1][:10], pybdsf_flux1_err[argsort1][:10],)
print()
print(tgss_flux[argsort1][:10], tgss_flux_err[argsort1][:10],)

[19.07944087 10.08407524  6.2040596   4.66649829  4.07043437  2.63676615
  2.59285157  2.32029264  2.01481443  2.00971007] [0.03169417 0.1228052  0.07585994 0.01961611 0.04961429 0.05010357
 0.01047804 0.02858712 0.01883139 0.04687232]

[22.3092 11.8522 11.8522  4.4188  3.8508  2.4844  3.0099  2.0635  1.9306
  1.6984] [2.2311 1.1857 1.1857 0.4423 0.3851 0.249  0.3012 0.2064 0.1931 0.17  ]


In [13]:
argsort1 = np.argsort(-pybdsf_flux1)
print(pybdsf_flux1[argsort1][:10], pybdsf_flux1_err[argsort1][:10],)
print()
print(tgss_flux[argsort1][:10], tgss_flux_err[argsort1][:10],)

[19.07944087 10.08407524  6.2040596   4.66649829  4.07043437  2.63676615
  2.59285157  2.32029264  2.01481443  2.00971007] [0.03169417 0.1228052  0.07585994 0.01961611 0.04961429 0.05010357
 0.01047804 0.02858712 0.01883139 0.04687232]

[22.3092 11.8522 11.8522  4.4188  3.8508  2.4844  3.0099  2.0635  1.9306
  1.6984] [2.2311 1.1857 1.1857 0.4423 0.3851 0.249  0.3012 0.2064 0.1931 0.17  ]


# Spectral Index

## NVSS

In [14]:
names = ['nvss', 'pybdsf']
search_radius = 20 * 3600 * u.arcsec
# flux_thresh = 2 # Jy
# # Set matching radius (e.g., 45 arcseconds for GLEAM-X resolution)
# match_radius = beam_size * u.arcsec 
# match_radius = 30 * u.arcsec # sqrt(40*22)

(pybdsf_ra2, pybdsf_dec2, nvss_ra, nvss_dec, pybdsf_flux2, 
 nvss_flux, pybdsf_flux2_err, nvss_flux_err) = find.matched_sources_nvss(nvss_catalog, pybdsf_catalog, flux_thresh*1e-5,
                                                                         target_ra_hms, target_dec_dms,search_radius, match_radius, names)
fac_nvss = 1. 
nvss_flux *= (fac_nvss)
nvss_flux_err *= (fac_nvss)


In [15]:
nu_nvss = 1.4e3 # MHz
nu_pybdsf = 147.4 # MHz

In [16]:
pybdsf_ra2.shape, nvss_ra.shape

((620,), (620,))

In [17]:
argsort2 = np.argsort(-pybdsf_flux2)
print(pybdsf_flux2[argsort2][:10], pybdsf_flux2_err[argsort2][:10],)
print()
print(nvss_flux[argsort2][:10], nvss_flux_err[argsort2][:10],)

[19.07944087 10.08407524  6.2040596   4.66649829  4.07043437  2.63676615
  2.59285157  2.32029264  2.01481443  2.00971007] [0.03169417 0.1228052  0.07585994 0.01961611 0.04961429 0.05010357
 0.01047804 0.02858712 0.01883139 0.04687232]

[3.7516 1.6939 1.2806 1.0166 0.9762 0.4396 0.3455 0.4168 0.5374 0.4374] [0.13   0.0558 0.0469 0.0356 0.0293 0.0155 0.012  0.0125 0.0189 0.0131]


In [18]:
alpha_nvss = ( np.log(nvss_flux) - np.log(pybdsf_flux2) ) / ( np.log(nu_nvss) - np.log(nu_pybdsf) ) 

In [19]:
alpha_nvss[argsort2][:10]

array([-0.7225113 , -0.79247555, -0.70094215, -0.67698478, -0.63429069,
       -0.79581589, -0.89535827, -0.76267569, -0.58706982, -0.67740787])

# Generate the Table

In [20]:
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u

# === Example arrays (replace with your real ones) ===
ids = np.array([329, 584, 585, 558, 1, 604, 89, 145, 238, 376])
gleam_ids_full = name[argsort_gleam][:10]


gmrt_flux = pybdsf_flux[argsort_gleam][:10]
gmrt_flux_err = pybdsf_flux_err[argsort_gleam][:10]

gleam_flux = gleam_flux_151[argsort][:10]
gleam_flux_err = gleam_flux_151_err[argsort][:10]

gleamx_flux = gleamx1_flux_151[argsortx1][:10]
gleamx_flux_err = gleamx1_flux_151_err[argsortx1][:10]
tgss_flux1 = tgss_flux[argsort1][:10]
tgss_flux_err1 = tgss_flux_err[argsort1][:10]

# spectral indices
alpha_gleam = alpha[argsort_gleam][:10]
alpha_gleam_err = alpha_err[argsort_gleam][:10]
alpha_gleamx = alphax1[argsortx1][:10]
alpha_gleamx_err = alphax1_err[argsortx1][:10]
alpha_nvss1 = alpha_nvss[argsort2][:10]

# RA/Dec in sexagesimal with ":" separator
coords = SkyCoord(ra=pybdsf_ra[argsort_gleam][:10]*u.deg,
                  dec=pybdsf_dec[argsort_gleam][:10]*u.deg)
ra_sex = coords.ra.to_string(unit=u.hour, sep=":", precision=2, pad=True)
dec_sex = coords.dec.to_string(unit=u.deg, sep=":", precision=2, pad=True, alwayssign=True)

# === Generate LaTeX table ===
print(r"\begin{table*}")
print(r"    \centering")
print(r"    \renewcommand{\arraystretch}{1.2} % Adjust row height")
print(r"    \setlength{\tabcolsep}{4pt}")
print(r"    \caption{The top 10 brightest sources from the uGMRT catalogue, sorted in descending order by total flux density, with the cross-matched sources from other catalogues. The columns show source IDs from our catalogue and GLEAM, the RA, Dec values in sexagesimal, and total flux densities with uncertainties at $\sim150$~MHz from uGMRT, GLEAM, GLEAM--X, and TGSS. The last three columns show the spectral indices quoted in the GLEAM, GLEAM--X, and those derived using our catalogue by comparing with the 1.4~GHz NVSS catalogue (Section~\ref{sec:spindex}).}")
print(r"    \label{tab:gmrt_gleam_nvss_alpha}")
print(r"    \begin{tabular}{cccccccccccc}")
print(r"    \hline")
print(r"    ID & GLEAM ID & RA & Dec & $S_{\rm int}$ & $S_{\rm int, GLEAM}$  & $S_{\rm int, GLEAM-X}$& $S_{\rm int, TGSS}$ & $\alpha_{\rm GLEAM}$ & $\alpha_{\rm GLEAM-X}$ & $\alpha_{\rm NVSS}$ \\")
print(r"     &  & (hh:mm:ss) & (dd:mm:ss) & Jy & Jy & Jy & Jy &  &  &  \\")
print(r"    \hline")

# RA/Dec in sexagesimal without fractional seconds
ra_sex = coords.ra.to_string(unit=u.hour, sep=":", precision=0, pad=True)
dec_sex = coords.dec.to_string(unit=u.deg, sep=":", precision=0, pad=True, alwayssign=True)

for i in range(10):
    # Clean GLEAM ID and replace '-' with '--'
    gleam_id = str(gleam_ids_full[i]).replace("GLEAM ", "").replace("-", "--")
    
    # flux with error (double backslash for LaTeX)
    s_gmrt = f"${gmrt_flux[i]:.2f} \\pm {gmrt_flux_err[i]:.2f}$"
    s_gleam = f"${gleam_flux[i]:.2f} \\pm {gleam_flux_err[i]:.2f}$"
    s_gleamx = f"${gleamx_flux[i]:.2f} \\pm {'-' if gleamx_flux_err[i] < 0 else f'{gleamx_flux_err[i]:.2f}'}$"
    s_tgss = f"${tgss_flux1[i]:.2f} \\pm {tgss_flux_err1[i]:.2f}$"

    # spectral indices
    a_gleam = f"${alpha_gleam[i]:.2f} \\pm {alpha_gleam_err[i]:.2f}$"
    a_gleamx = "--" if np.isnan(alpha_gleamx[i]) else f"${alpha_gleamx[i]:.2f} \\pm {alpha_gleamx_err[i]:.2f}$"
    a_nvss = f"${alpha_nvss1[i]:.2f}$"

    # replace any '-' in dec with '--' for LaTeX
    ra_print = ra_sex[i].replace("-", "--")
    dec_print = dec_sex[i].replace("-", "--")

    print(f"    {ids[i]}  & {gleam_id} & {ra_print} & {dec_print} & {s_gmrt} & {s_gleam} & {s_gleamx} & {s_tgss} & {a_gleam} & {a_gleamx} & {a_nvss} \\\\[3pt]")

print(r"    \hline")
print(r"    \end{tabular}")
print(r"\end{table*}")


\begin{table*}
    \centering
    \renewcommand{\arraystretch}{1.2} % Adjust row height
    \setlength{\tabcolsep}{4pt}
    \caption{The top 10 brightest sources from the uGMRT catalogue, sorted in descending order by total flux density, with the cross-matched sources from other catalogues. The columns show source IDs from our catalogue and GLEAM, the RA, Dec values in sexagesimal, and total flux densities with uncertainties at $\sim150$~MHz from uGMRT, GLEAM, GLEAM--X, and TGSS. The last three columns show the spectral indices quoted in the GLEAM, GLEAM--X, and those derived using our catalogue by comparing with the 1.4~GHz NVSS catalogue (Section~\ref{sec:spindex}).}
    \label{tab:gmrt_gleam_nvss_alpha}
    \begin{tabular}{cccccccccccc}
    \hline
    ID & GLEAM ID & RA & Dec & $S_{\rm int}$ & $S_{\rm int, GLEAM}$  & $S_{\rm int, GLEAM-X}$& $S_{\rm int, TGSS}$ & $\alpha_{\rm GLEAM}$ & $\alpha_{\rm GLEAM-X}$ & $\alpha_{\rm NVSS}$ \\
     &  & (hh:mm:ss) & (dd:mm:ss) & Jy & Jy & Jy & 